In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup as bs
import lxml
import glob
import re
import json

In [2]:
def clean_section(s):
    has_paragraph = False
    place_title = ""
    clean_lines = []
    lines = s.split("\n")
    for line in lines:
            
        if re.search("s[0-9]+\">", line): # If line does not contain sth like: s1">
            line = re.sub("s[0-9]+\">(.*)", r"\1", line)
            line = line.strip()
            if line:
                clean_lines.append(line)
        if re.search("</section>", line): # If line does not contain sth like: s1">
            line = line.strip()
            line = re.sub("(.*)</section>", r"\1", line)
            if line:
                clean_lines.append(line)
        else:
            clean_lines.append(line)
                
    lines = clean_lines
    clean_lines = []
                
    for line in lines:
        line = line.strip()
        if not re.match("s[0-9]+\">", line) and not line == "</section>": # If line does not have this shape: s1">
            if line.startswith("<para"):
                has_paragraph = True
                clean_lines.append(line)
            else:
#                 if re.match(".*<para", line) and has_paragraph == False:
#                     print(re.sub("(.*)<para", r"\1", line))
                    
                if has_paragraph == False:
                    place_title = line
                else:
                    clean_lines.append(line)
    clean_text = " ".join(clean_lines)
    # clean_text = bs(clean_text).get_text() # Return clean text
    return place_title, clean_text

In [3]:
regex_section = r"s[0-9]+\">.*"

entry_id = 0
bho_dict = dict()
for path in glob.glob("/resources/bho/*/*"):
    # Read the XML file
    with open(path, "r", encoding="ISO-8859-1") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a stringx
        content = "".join(content)
        bs_content = bs(content, "lxml")
        report_title = bs_content.html.body.find("title").getText()
        if report_title != "Preface":
            sectext = str(bs_content.html.body)
            sections = sectext.split("<section id=\"")
            for s in sections:
                s = s.strip()
                if re.match(regex_section, s):
                    entry_id += 1
                    title, text = clean_section(s)
                    bho_dict[entry_id] = {"report_title": report_title,
                                          "place_name": title,
                                          "description": text,
                                          "filepath": path}

In [4]:
with open('bho_parsed.json', 'w') as f:
    json.dump(bho_dict, f)

In [5]:
with open('bho_parsed.json') as f:
    my_dict = json.load(f)
    for k in list(my_dict.keys())[:5]:
        print(k)
        print(my_dict[k])
        print("=========")

1
{'report_title': 'Abbas-Combe - Aberystwith', 'place_name': 'A TOPOGRAPHICAL DICTIONARY OF ENGLAND.', 'description': '', 'filepath': '/resources/bho/England/50741.xml'}
2
{'report_title': 'Abbas-Combe - Aberystwith', 'place_name': 'Abbas-Combe, or Temple-Combe (St. Mary)', 'description': '<para id="p1">ABBAS-COMBE, or Temple-Combe (<emph type="i">St. Mary</emph>), a parish, in the union of Wincanton, hundred of Horethorne, E. division of Somerset, 4½ miles (S. by W.) from Wincanton, on the road to Blandford; containing 461 inhabitants. It derived the name of Temple-Combe from the military order of Knights Templars, who had an establishment here, which at the Dissolution possessed a revenue of £128. 7. 9. Some remains of the chapel attached to the old priory-house are still to be seen. The parish comprises by measurement 1884 acres of land; and contains good building-stone of the granite species, and limestone, both of which are quarried. The living is a rectory, valued in the king\'s